In [1]:
import json
import os
import ase.io

from ase.io               import read,write
from ase.io               import vasp
from hiphive              import ClusterSpace, StructureContainer, ForceConstantPotential
from hiphive.utilities    import prepare_structures,get_displacements
from trainstation         import Optimizer
from ase.calculators.vasp import Vasp
from ase.io.vasp          import read_vasp,write_vasp

Read structures containing displacements and forces

In [2]:
root_path = f'data/CrN-riccardo/all-volumes'

# Create and save as a dictionary
setup_variables = {
    'root_path':                   root_path,
    'path_to_relaxations':         f'{root_path}/calculations',
    'path_to_primtive_cell':       f'{root_path}/POSCAR_unit',
    'path_to_super_cell':          f'{root_path}/POSCAR_supercell',
    'path_to_cluster_space':       f'{root_path}/cluster_space.cs',
    'path_to_structure_container': f'{root_path}/structure_container.sc',
    'path_to_fcp':                 f'{root_path}/material.fcp'
}

# Write the dictionary to the file in JSON format
with open(f'setup_variables.json', 'w') as json_file:
    json.dump(setup_variables, json_file)

In [3]:
# Cutoffs for pairs, triplets and quadruplets (in angstrom) 
cutoffs = [4.0, 4.0, 3.0]

In [4]:
primtive_cell = read_vasp(setup_variables['path_to_primtive_cell'])
super_cell    = read_vasp(setup_variables['path_to_super_cell'])

rattled_structures = []
for folder in os.listdir(setup_variables['path_to_relaxations']):
    print(folder)
    path_to_vasprun = f"{setup_variables['path_to_relaxations']}/{folder}/vasprun.xml.gz"
    
    # Read vasprun
    if os.path.exists(path_to_vasprun):
        # Read xml file with forces
        temp_vasprun = ase.io.read(path_to_vasprun)

        # Append to main list
        rattled_structures.append(temp_vasprun)

disp-0010-SC_2x2x2_Vol_75.18
disp-0008-SC_2x2x2_Vol_75.18
disp-0001-SC_2x2x2_Vol_75.18
disp-0008-SC_2x2x2_Vol_76.69


In [5]:
print(rattled_structures)

[Atoms(symbols='Cr32N32', pbc=True, cell=[8.41723758, 8.45317928, 8.45317928], calculator=SinglePointDFTCalculator(...)), Atoms(symbols='Cr32N32', pbc=True, cell=[8.41723758, 8.45317928, 8.45317928], calculator=SinglePointDFTCalculator(...)), Atoms(symbols='Cr32N32', pbc=True, cell=[8.41723758, 8.45317928, 8.45317928], calculator=SinglePointDFTCalculator(...)), Atoms(symbols='Cr32N32', pbc=True, cell=[8.4729825, 8.50916223, 8.50916223], calculator=SinglePointDFTCalculator(...))]


Set up cluster space

In [6]:
cs = ClusterSpace(primtive_cell, cutoffs)

# Save ClusterSpace
cs.write(setup_variables['path_to_cluster_space'])
print(cs)

Primitive cell:
    Formula: CrN
    Cell:
        [  2.10431   2.11329   0.00000]
        [  2.10431  -2.11329   0.00000]
        [ -2.10431   0.00000  -2.11329]
    Basis:
        Cr  [  0.50000   0.50000   0.00000]
        N   [  0.00000   0.00000   0.00000]

Crystal symmetry:
    Spacegroup:          I4/mmm (139)
    Unique site:         2
    Symmetry operations: 16
    symprec:             1.00e-05

Cutoffs:
    Maximum cutoff: 4.0
    Found 2 center atoms with 34 images totaling 36 atoms

Clusters:
    Clusters: {2: 53, 3: 352, 4: 535}
    Total number of clusters: 940

Orbits:
    Orbits: {2: 9, 3: 22, 4: 36}
    Total number of orbits: 67

Eigentensors:
    Eigentensors: {2: 28, 3: 220, 4: 728}
    Total number of parameters: 976
    Discarded orbits:
        (0, 0, 0)
        (1, 1, 1)

Constraints:
    Acoustic: True
    Number of degrees of freedom: {2: 24, 3: 183, 4: 409}
    Total number of degrees of freedom: 616
=================== Cluster Space ====================
Spa

In [7]:
cs.print_orbits()

===================================== List of Orbits =====================================
index | order |      elements      |  radius  |     prototype      | clusters | parameters
------------------------------------------------------------------------------------------
  0   |   2   |       Cr Cr        |  0.0000  |       (0, 0)       |    1     |     2     
  1   |   2   |        Cr N        |  1.0522  |       (0, 1)       |    2     |     2     
  2   |   2   |       Cr Cr        |  1.4912  |       (0, 2)       |    4     |     4     
  3   |   2   |        Cr N        |  1.8276  |       (0, 3)       |    8     |     5     
  4   |   2   |        Cr N        |  1.0566  |       (0, 5)       |    4     |     3     
  5   |   2   |       Cr Cr        |  1.4943  |       (0, 6)       |    2     |     3     
  6   |   2   |        N N         |  0.0000  |       (1, 1)       |    1     |     2     
  7   |   2   |        N N         |  1.4912  |       (1, 5)       |    4     |     4     

Set up structure container

In [8]:
structures = prepare_structures(rattled_structures, super_cell)
sc = StructureContainer(cs)
for structure in structures:
    sc.add_structure(structure)

# Save StructureContainer
sc.write(setup_variables['path_to_structure_container'])
print(sc)

AssertionError: 

Train the model

In [ ]:
opt = Optimizer(sc.get_fit_data())
opt.train()
print(opt)

Construct force constant potential

In [ ]:
fcp = ForceConstantPotential(cs, opt.parameters)

# Save ForceConstantPotential
fcp.write(setup_variables['path_to_fcp'])
print(fcp)